This script will read in the tif files and split into chunks of 128x128 for training the model.  Note I will ignore all chunks where there is no data for dNBR as we wouldn't want to train on this.  
The in path and outpath have to be where the files from the tif files were sent to from the google cloud storage bucket after downloading from earth engine. 

Read in packages

In [1]:
import pandas as pd
import os
import numpy as np
import xarray as xr
import rioxarray
import glob
import random
import geopandas as gpd
from sklearn.utils import shuffle
from MightyMosaic import MightyMosaic


First take the input tif files and chunk them to sizes of 128x128 for the neural network.  

In [2]:
"""arguments are path to .tif files to be chunked, out_path to save files, chunk size, and if desired a threshold"""       

#check if all 0
def is_matrix_all_zeros(matrix):
    # Convert the matrix to a NumPy array
    np_matrix = np.array(matrix)

    # Check if all elements in the array are zeros
    return np.all(np_matrix == 0)

# #in_path to tif files
# in_path = '/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/modis_nbac'

# #out_path which will be manipulated based on parameters below
# out_path = '/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/modis_nbac'


#in_path to tif files
in_path = '/explore/nobackup/people/spotter5/cnn_mapping/Russia/modis'

#out_path which will be manipulated based on parameters below
out_path = '/explore/nobackup/people/spotter5/cnn_mapping/Russia/modis'


#right now everything is set to use 0 dnbr threshold
size = 16
threshold = True

if threshold == True:

    out_path = out_path + '_subs_0_' + str(size)
    if not os.path.isdir(out_path):
        os.makedirs(out_path)

if threshold == False:

    out_path = out_path + '_subs_' + str(size)
    if not os.path.isdir(out_path):
        os.makedirs(out_path)

files = os.listdir(in_path)

# files = ['median_6169.tif']

bad = []
#note bands for MODIS are different, we have more.  THey are red, NIR1, blue/green, green, NIR2, SWIR1, SWIR2, dNBR, dNDVI, dNDII, y. 

#loop through files
for f in files:

    if f.endswith('.tif'):
        
         
        # #in file
        try:
        
            #file name
            f2 = f.replace('median_', '')

            #ID of file
            f_id = int(f2.replace('.tif', ''))



            #read in file
            in_mat = rioxarray.open_rasterio(os.path.join(in_path, f))


            #now turn in mat to numpy, and in mtbs to numpy
            in_mat = in_mat.to_numpy().astype(float)


            #convert to band last
            in_mat = np.moveaxis(in_mat, 0, 2) 


            #get dnbr
            t = in_mat[:, :, 7]

             #check if all 0
            result = is_matrix_all_zeros(t)

            #if it is not all 0s keep data
            if result == False:

                #mosaic
                mosaic = MightyMosaic.from_array(in_mat, (size,size), overlap_factor=1) 

                #take off last dimensions of mosaic which give edge effects as they are filled with no data
                mosaic = mosaic[:-1, :-1, :, :, :]


                #first two dimensions are number of chunks, 3 and 4 are size (256, 266) and last is nbands
                for i in range(mosaic.shape[0]):

                    for j in range(mosaic.shape[1]):

                        fname = os.path.join(str(i) + '_' +  str(j)+ '_' + f2.replace('.tif', '.npy'))
                        out_name = os.path.join(out_path, fname)

                        if os.path.exists(out_name) == False:

                            in_mat3 = mosaic[i, j, :, :, :-1]

                            target = mosaic[i, j, :, :, 10]

                            #turn nan to -999
                            in_mat[np.isnan(in_mat)] = -999

                            target = target.astype('int')

                            target[target <0 ] = 0
                            target[target >1 ] = 0

                            #
                            target[~np.isin(target, [0,1])] = 0

                            #turn nan to 0
                            target[np.isnan(target)] = 0

                            #if the target is all 0 don't train on it
                            # if np.all(target == 0) == False:

                            #now get dnbr which is the 7th band
                            dnbr = in_mat3[:, :, 7]

                            result1 = is_matrix_all_zeros(dnbr)

                            if result1 == False:

                                #if threshold apply dnbr threshold
                                target[dnbr < 0] = 0

                                e = np.dstack([in_mat3, target])

                                np.save(out_name, e)

                                print(f)
                                
                            if result1 == True:
                                bad.append(f)

            if result == True:
                bad.append(f)

        except:
            # bad.append(f)
            pass

median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4224.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4218.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4236.tif
median_4

In [3]:
't'

't'

Now take those files and save a csv with file names for the 80/10/10 training, validation, testing parts

In [4]:

def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

#list of files
# # file_names = listdir_fullpath('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/nbac_1985_sent_harm_subs_128')
# file_names = listdir_fullpath('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/modis_nbac_subs_0_32')
file_names = listdir_fullpath('/explore/nobackup/people/spotter5/cnn_mapping/Russia/modis_subs_0_16')

#shuffle file_names
file_names = shuffle(file_names, random_state = 555)


print(len(file_names))

random.seed(555)
# #get train at 80%

train_files = random.sample(file_names, k=round(len(file_names) * 0.8))

#files at 10%
val_files = list(set(file_names) - set(train_files))
val_files = random.sample(val_files, k=round(len(file_names) * 0.1))

#combine traini and val
temp = train_files + val_files
#get test files at 10%
test_files = list(set(file_names) - set(temp))


#convert the lists to pandas dataframes
train = pd.DataFrame({'Files': train_files})
val = pd.DataFrame({'Files': val_files})
test = pd.DataFrame({'Files': test_files})

print(train.shape)
print(val.shape)
print(test.shape)

# train.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/modis_nbac_32_training_files.csv')
# val.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/modis_nbac_32_validation_files.csv')
# test.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/modis_nbac_32_testing_files.csv')

train.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/modis_16_training_files.csv')
val.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/modis_16_validation_files.csv')
test.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/modis_16_testing_files.csv')


240849
(192679, 1)
(24085, 1)
(24085, 1)


In [7]:
't'

't'